In [22]:
pip install langchain-groq


Note: you may need to restart the kernel to use updated packages.


In [23]:
pip install chromadb

Note: you may need to restart the kernel to use updated packages.


In [24]:
from langchain_groq import ChatGroq


In [25]:

llm = ChatGroq(
    temperature=0,
    groq_api_key="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", 
    model_name="llama-3.3-70b-versatile"
)

response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [26]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://amazon.jobs/en/jobs/3132857/agentic-ai-teacher")
page_data = loader.load().pop().page_content
print(page_data)

Agentic AI Teacher - Job ID: 3132857 | Amazon.jobs
Skip to main content×HomeTeamsLocationsJob categoriesMy careerMy applicationsMy profileAccount securitySettingsSign outResourcesAccommodationsBenefitsInclusive experiencesHow We HireLeadership principlesWorking at AmazonFAQAgentic AI TeacherJob ID: 3132857 | ADCI MAA 12 SEZApply nowDescriptionAmazon is investing in generative AI and the responsible development and deployment of Large Language Models (LLMs) across all our businesses. As part of the Data Team, you will deliver high-quality training data to improve and expand our proprietary LLM capabilities. We seek innovative candidates who can solve complex problems and drive technological advancement in AI. Your work will directly impact how Amazon's customers interact with our AI systems across multiple business lines.This is a fixed term contractual role for 1 year, requiring flexibility to work in rotational shifts.Key job responsibilities- Create and annotate high-quality complex 

In [27]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [28]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Agentic AI Teacher',
 'experience': "Bachelor's degree or equivalent",
 'skills': ['Advanced English-level language proficiency (C1+ or equivalent fluency by Common European Framework of Reference for Languages (CEFR) standards)',
  'Python',
  'SQL',
  'Annotation',
  'Data markup',
  'Stakeholder relationship management'],
 'description': 'Create and annotate high-quality complex training data in multiple modalities (text, image, video) on various topics, including technical or science-related content. Write grammatically correct texts in different styles with various degrees of creativity, strictly adhering to provided guidelines. Interpret technical documentation to implement solutions accurately. Dive deep into issues and implement solutions independently. Identify and report tooling bugs and suggest improvements. Make sound judgments and logical decisions when faced with ambiguous or incomplete information.'}

In [29]:
type(json_res)

dict

In [30]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [31]:

import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [34]:
job = json_res

In [35]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/wordpress-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}]]

In [36]:
job['skills']

['Advanced English-level language proficiency (C1+ or equivalent fluency by Common European Framework of Reference for Languages (CEFR) standards)',
 'Python',
 'SQL',
 'Annotation',
 'Data markup',
 'Stakeholder relationship management']

In [38]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are  Zaid, a business development executive at FangZZ.  FangZZ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase  FangZZ's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert AI Solutions for Agentic AI Teacher Role

Dear Hiring Manager,

I came across the job description for the Agentic AI Teacher role and was impressed by the complexity and scope of the project. As a Business Development Executive at FangZZ, I believe our team has the expertise and capabilities to fulfill your requirements.

Our team at FangZZ has extensive experience in creating and annotating high-quality training data in multiple modalities, including text, image, and video. We have a strong proficiency in Python, SQL, and data markup, which aligns with your technical requirements. Our expertise in stakeholder relationship management ensures seamless communication and collaboration throughout the project.

We have a proven track record of delivering tailored solutions that foster scalability, process optimization, cost reduction, and heightened overall efficiency. Our portfolio includes successful projects in machine learning and Python development, which can be viewed 